# установка библиотек 

In [26]:
!pip install pandas 
!pip install -U scikit-learn

импортирую библиотеки 

In [27]:
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

загружаю csv файл 

In [28]:
data=pd.read_csv('/home/user1/Desktops/Desktop1/сессии/visitation.csv')
data.head()#вывожу таблицу

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,Russia,iPhone,organic,2022-05-01 02:36:01,2022-05-01 02:45:01
1,278965908054,Russia,iPhone,organic,2022-05-01 04:46:31,2022-05-01 04:47:35
2,590706206550,Russia,Mac,organic,2022-05-01 14:09:25,2022-05-01 15:32:08
3,326433527971,Russia,Android,TipTop,2022-05-01 00:29:59,2022-05-01 00:54:25
4,349773784594,Russia,Mac,organic,2022-05-01 03:33:35,2022-05-01 03:57:40


перекодирую данные

In [29]:
# Подключаем класс для предобработки данных
from sklearn import preprocessing

# Напишем функцию, которая принимает на вход DataFrame, кодирует числовыми значениями категориальные признаки
# и возвращает обновленный DataFrame и сами кодировщики.
def number_encode_features(init_df):
    result = init_df.copy() # копируем нашу исходную таблицу
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == object: # np.object -- строковый тип / если тип столбца - строка, то нужно его закодировать
            encoders[column] = preprocessing.LabelEncoder() # для колонки column создаем кодировщик
            result[column] = encoders[column].fit_transform(result[column]) # применяем кодировщик к столбцу и перезаписываем столбец
    return result, encoders

df_orders, encoders = number_encode_features(data) # Теперь encoded data содержит закодированные кат. признаки
df_orders.head()

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,1,3,10,99,85
1,278965908054,1,3,10,169,159
2,590706206550,1,1,10,508,546
3,326433527971,1,0,6,27,24
4,349773784594,1,1,10,128,130


#  4.1 Предсказание целевой переменной

задаю переменной X мои перекодированные данные

In [30]:
X=df_orders
X.head()

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,1,3,10,99,85
1,278965908054,1,3,10,169,159
2,590706206550,1,1,10,508,546
3,326433527971,1,0,6,27,24
4,349773784594,1,1,10,128,130


присваиваю переменной y колонну Channel

In [31]:
y=df_orders['Channel']



разделяем данные на обучающую и тестовую выборки

test_size=0.2 задает размер тестовой выборки в 20% от исходных данных.

random_state=42 задает начальное значение для генератора случайных чисел, чтобы результаты можно было воспроизвести.


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

обучаю модель 

In [33]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Оценка R2 – очень важный показатель, который используется для оценки производительности модели машинного обучения на основе регрессии.

In [34]:
r2=r2_score(model.predict(X_test), y_test)
print(f'Производительность модели: {r2}')

Производительность модели: 1.0


присваиваем переменной y_pred значения для вывода точности модели 

In [35]:
y_pred = model.predict(X_test)

Accuracy — это метрика, которая описывает общую точность предсказания модели по всем классам.


In [36]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0


присваию col имена столбцов 

In [37]:
col=['User Id', 'Region', 'Device', 'Channel', 'Session Start',
       'Session End']

вывожу значимость признаков 

In [38]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

ar_f=[]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), col[idx]])
print("Значимость признака:")
ar_f.sort(reverse=True)
ar_f

Значимость признака:


[[0.8782, 'Channel'],
 [0.0953, 'Region'],
 [0.0082, 'Session End'],
 [0.0076, 'Session Start'],
 [0.0062, 'User Id'],
 [0.0045, 'Device']]

**Я провел обучение модели и так же использовал R2 метрику**

R2 показала ппроизводительность модели : 1.0

Я вывел значимость каждого атрибута в моем датафрейме , самым значимым признаком оказался  Channel 

Выполнил прогнозирование для контрольной выборки с

помощью полученной модели. Определил качество модели на контрольной

выборке с помощью коэффициента детерминации (R2).


# 4.2 Разработка прикладного решения

In [39]:
import pandas as pd

# Загрузка CSV-файла
data = pd.read_csv('/home/user1/Desktops/Desktop1/сессии/visitation.csv')

# Преобразование данных в словарь
data_dict = data.to_dict(orient="records")

In [40]:
!pip install flask

In [44]:
data.head()

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,Russia,iPhone,organic,2022-05-01 02:36:01,2022-05-01 02:45:01
1,278965908054,Russia,iPhone,organic,2022-05-01 04:46:31,2022-05-01 04:47:35
2,590706206550,Russia,Mac,organic,2022-05-01 14:09:25,2022-05-01 15:32:08
3,326433527971,Russia,Android,TipTop,2022-05-01 00:29:59,2022-05-01 00:54:25
4,349773784594,Russia,Mac,organic,2022-05-01 03:33:35,2022-05-01 03:57:40


In [46]:
import pandas as pd
from flask import Flask, request, jsonify
import pickle

# Загрузка модели машинного обучения
model = data

# Создание приложения Flask
app = Flask(__name__)

# Маршрут для метода predict
@app.route("/predict", methods=["POST"])
def predict():
    # Получение данных из запроса
    data = request.get_json()

    # Формирование признаков для прогноза
    features = [data["Channel"], data["Region"]]

    # Генерация прогноза
    prediction = model.predict([features])

    # Возврат прогноза
    return jsonify({"прогноз": prediction[0]})

# Запуск приложения
if __name__ == "__main__":
    app.run(host="localhost", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Mar/2024 14:36:56] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [05/Mar/2024 14:36:56] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [05/Mar/2024 14:36:56] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [05/Mar/2024 14:36:56] "GET /favicon.ico HTTP/1.1" 404 -


# Отчет 

**В этой сессии я прошел обучение выбранное модели**

 и так же выполнил R2 метрику , и я вывел значимость каждого атрибута в моем датафрейме , самым значимым признаком оказался  Channel .


Выполнил прогнозирование для контрольной выборки с

помощью полученной модели. Определил качество модели на контрольной

выборке с помощью коэффициента детерминации (R2).

**Разработка API**


я считаю что у меня не получилось ее сделать , так как было не достаточно времени , и у меня не получилось сделать базу данных 